vgg + adam 모델 튜닝    - 오늘 5시 50분

모델 붙이기             - 내일 오전    ==> 디코딩 구현, //test data 출력//

ppt 자료, 발표 준비 //  - 내일 오전 

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import pandas as pd

In [18]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA(GPU)를 사용합니다.")
else:
    device = torch.device("cpu")
    print("CUDA(GPU)를 사용할 수 없으므로, CPU를 사용합니다.")

CUDA(GPU)를 사용할 수 없으므로, CPU를 사용합니다.


In [19]:
# CIFAR10 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize([0.6580, 0.6161, 0.5856], [0.2715, 0.2830, 0.2974]) # 알려진 CIFAR10의 평균과 표준편차
])

# 데이터셋 불러오기
garbage_dataset = datasets.ImageFolder(root="./archive02/garbage_classification", transform=transform)
# DataLoader를 사용하여 이미지 데이터를 배치 단위로 불러올 수 있음
# garbage_loader = torch.utils.data.DataLoader(garbage_dataset, batch_size=1, shuffle=False)

total_size = len(garbage_dataset)
# 훈련 세트의 비율 설정
train_ratio = 0.8

# 훈련 세트와 테스트 세트로 나누기
train_size = int(train_ratio * total_size)
test_size = total_size - train_size
train_dataset, test_dataset = torch.utils.data.random_split(garbage_dataset, [train_size, test_size])

# 훈련과 테스트 데이터로더 생성
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip

transform_original = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4913, 0.4821, 0.4465], [0.2470, 0.2434, 0.2615])
])

transform_horizontal_flip = transforms.Compose([
    RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4913, 0.4821, 0.4465], [0.2470, 0.2434, 0.2615])
])

transform_vertical_flip = transforms.Compose([
    RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4913, 0.4821, 0.4465], [0.2470, 0.2434, 0.2615])
])

train_dataset_original = datasets.CIFAR10('./CIFAR10', train=True, download=True, transform=transform_original)
train_dataset_horizontal_flip = datasets.CIFAR10('./CIFAR10', train=True, download=True, transform=transform_horizontal_flip)
train_dataset_vertical_flip = datasets.CIFAR10('./CIFAR10', train=True, download=True, transform=transform_vertical_flip)

# 결합
from torch.utils.data import ConcatDataset
combined_dataset = ConcatDataset([train_dataset_original, train_dataset_horizontal_flip, train_dataset_vertical_flip])

# 데이터 로더
combined_loader = torch.utils.data.DataLoader(combined_dataset, batch_size=128, shuffle=True)

In [20]:
### basic ###
class myVggNet(nn.Module):
    def __init__(self):
        super(myVggNet, self).__init__()
        # vgg16 모델 로드
        self.vggnet = models.vgg19()
        # 출력 레이어를 클래스 수에 맞게 조정
        num_ftrs = self.vggnet.classifier[6].in_features
        self.vggnet.classifier[6].in_features = nn.Linear(num_ftrs, 12)

    def forward(self, x):
        return self.vggnet(x)

In [23]:
# 훈련 함수    

def train(model, train_loader, optimizer, epoch):
    
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for data, target in train_loader:
        if torch.cuda.is_available():
            data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # 예측값 계산(가장 높은 값을 가진 인덱스)
    _, predicted = torch.max(output.data, 1)
    total += target.size(0)
    # 올바르게 예측된 샘플의 수 업데이트
    correct += (predicted == target).sum().item()

    # 평균 훈련 손실 계산
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    print(f'Epoch {epoch}, Training loss: {train_loss:.4f}')
    accuracy = 100. * correct / total
    train_accuracies.append(accuracy)

In [24]:
# 테스트 함수
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if torch.cuda.is_available():
                data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    test_losses.append(test_loss)
    test_accuracies.append(accuracy)
    print(f'Test loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

# L2 0.001

In [21]:
# 손실과 정확도를 저장하기 위한 리스트 초기화
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

In [22]:
# batchsize 256, 128, lr: 0.0001
# vgg19
# optimizer: adam
# 훈련 및 테스트 실행


### L2 정규화 추가 ###

model = myVggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.001)

In [25]:
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Epoch 1, Training loss: 2.7221
Test loss: 1.9517, Accuracy: 38.48%
Epoch 2, Training loss: 1.8866
Test loss: 1.6668, Accuracy: 43.35%
Epoch 3, Training loss: 1.5846
Test loss: 1.3865, Accuracy: 48.98%
Epoch 4, Training loss: 1.3786
Test loss: 1.3031, Accuracy: 53.53%
Epoch 5, Training loss: 1.2062
Test loss: 1.1010, Accuracy: 62.71%


In [ ]:
import matplotlib.pyplot as plt

# 손실 및 정확도 시각화
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].plot(train_losses, label='Training Loss')
axs[0].plot(test_losses, label='Test Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].set_title('Training and Test Loss')

axs[1].plot(train_accuracies, label='Training Accuracy', color='blue')
axs[1].plot(test_accuracies, label='Test Accuracy', color='orange')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend()
axs[1].set_title('Training and Test Accuracy')

plt.tight_layout()
plt.show()

# L2 0.0005

In [ ]:
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

In [ ]:
model = myVggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.0005)

In [ ]:
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

In [ ]:
import matplotlib.pyplot as plt

# 손실 및 정확도 시각화
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].plot(train_losses, label='Training Loss')
axs[0].plot(test_losses, label='Test Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].set_title('Training and Test Loss')

axs[1].plot(train_accuracies, label='Training Accuracy', color='blue')
axs[1].plot(test_accuracies, label='Test Accuracy', color='orange')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend()
axs[1].set_title('Training and Test Accuracy')

plt.tight_layout()
plt.show()

# L2 0.00025

In [ ]:
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

In [ ]:
model = myVggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.00025)

In [ ]:
for epoch in range(1, 21):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

In [ ]:
import matplotlib.pyplot as plt

# 손실 및 정확도 시각화
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].plot(train_losses, label='Training Loss')
axs[0].plot(test_losses, label='Test Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].set_title('Training and Test Loss')

axs[1].plot(train_accuracies, label='Training Accuracy', color='blue')
axs[1].plot(test_accuracies, label='Test Accuracy', color='orange')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend()
axs[1].set_title('Training and Test Accuracy')

plt.tight_layout()
plt.show()

# L2 0.0001

In [ ]:
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

In [ ]:
model = myVggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.0001)

In [ ]:
import matplotlib.pyplot as plt

# 손실 및 정확도 시각화
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].plot(train_losses, label='Training Loss')
axs[0].plot(test_losses, label='Test Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].set_title('Training and Test Loss')

axs[1].plot(train_accuracies, label='Training Accuracy', color='blue')
axs[1].plot(test_accuracies, label='Test Accuracy', color='orange')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy (%)')
axs[1].legend()
axs[1].set_title('Training and Test Accuracy')

plt.tight_layout()
plt.show()